# Action recognition using TAO ActionRecognitionNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080">


## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Train 3D RGB only for action recognition on the subset of [HMDB51](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/) dataset.
* Evaluate the trained model.
* Run Inference on the trained model.
* Export the trained model to a .etlt file for deployment to DeepStream.

## Table of Contents

This notebook shows an example usecase of ActionRecognitionNet using Train Adapt Optimize (TAO) Toolkit.

1. [Set up env variables and map drives](#head-1)
2. [Prepare dataset and pre-trained model](#head-2)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Inferences](#head-6)


## Connect to a GPU Runtime

1.   Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accerlerator)
2.   Then click on Connect (Top Right)



## Mounting Google drive
Mount your Google drive storage to this Colab instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Setup Python Environment
Setup the environment necessary to run the TAO Networks by running the bash script

In [ ]:
!sh /content/drive/MyDrive/ColabNotebooks/pytorch/setup_env.sh

## 1. Set up env variables and map drives <a class="anchor" id="head-1"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The TAO launcher uses docker containers under the hood, and **for our data and results directory to be visible to the docker, they need to be mapped**. The launcher can be configured using the config file `~/.tao_mounts.json`. Apart from the mounts, you can also configure additional options like the Environment Variables and amount of Shared Memory available to the TAO launcher. <br>

`IMPORTANT NOTE:` The code below creates a sample `~/.tao_mounts.json`  file. Here, we can map directories in which we save the data, specs, results and cache. You should configure it for your specific case so these directories are correctly visible to the docker container.


In [ ]:
# defaulted to volatile colab instance memory
# depending on the available storage left between google drive and colab instance, choose the data_dir path
%env DATA_DIR=/content/data

%env LOCAL_PROJECT_DIR=/content

# note: You could set the HOST_SPECS_DIR to folder of the experiments specs downloaded with the notebook
%env SPECS_DIR=/content/drive/MyDrive/ColabNotebooks/pytorch/cv_notebooks/action_recognition_net/specs
%env RESULTS_DIR=/content/results

# Set your encryption key, and use the same key for all commands
%env KEY = nvidia_tao

In [ ]:
! mkdir -p $DATA_DIR
! mkdir -p $SPECS_DIR
! mkdir -p $RESULTS_DIR

## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

### 2.1 Prepare dataset

 We will be using the [HMDB51](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/) dataset for the tutorial. Download the HMDB51 dataset and unrar them firstly (We choose fall_floor/ride_bike for this tutorial): 

In [ ]:
# install unrar
# NOTE: The following commands require `sudo`. You can run the command outside the notebook.
!apt update
!apt-get install unrar

In [ ]:
# download the dataset and unrar the files
!wget -P $DATA_DIR http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
!mkdir -p $DATA_DIR/videos && unrar x -o+ $DATA_DIR/hmdb51_org.rar $DATA_DIR/videos
!mkdir -p $DATA_DIR/raw_data
!unrar x -o+ $DATA_DIR/videos/fall_floor.rar $DATA_DIR/raw_data
# Depending on the space available, delete some videos before unraring the next actions
!unrar x -o+ $DATA_DIR/videos/ride_bike.rar $DATA_DIR/raw_data

Clone the dataset process script

In [ ]:
%cd /content
!if [ -d tao_toolkit_recipes ]; then rm -rf tao_toolkit_recipes; fi
!git clone https://github.com/NVIDIA-AI-IOT/tao_toolkit_recipes

Install the dependency for data generator:

In [ ]:
!pip3 install xmltodict opencv-python

Run the process script. 

In [ ]:
%cd /content/tao_toolkit_recipes/tao_action_recognition/data_generation/
!bash ./preprocess_HMDB_RGB.sh $DATA_DIR/raw_data $DATA_DIR/processed_data

We also provide scripts to preprocess optical flow dataset. The following cells for processing optical flow dataset is `Optional`.

`OPTIONAL:` Download the app based on NVOF SDK to generate optical flow. It is packaged with this notebook.

In [ ]:
#!echo <passwd> | sudo -S apt install -y libfreeimage-dev

`OPTIONAL` Run the process script for HMDB. 

`IMPORTANT NOTE`: to run the `preprocess_HMDB.sh` generating optical flow, a Turing or Ampere above GPU is needed. 

In [ ]:
# !cp ./AppOFCuda tao_toolkit_recipes/tao_action_recognition/data_generation/
# %cd tao_toolkit_recipes/tao_action_recognition/data_generation/
# !bash ./preprocess_HMDB.sh $DATA_DIR/raw_data $DATA_DIR/processed_data

In [ ]:
# download the split files and unrar
!wget -P $DATA_DIR http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar
!mkdir -p $DATA_DIR/splits && unrar x -o+ $DATA_DIR/test_train_splits.rar $DATA_DIR/splits

In [ ]:
# run split_HMDB to generate training split
!if [ -d $DATA_DIR/train ]; then rm -rf $DATA_DIR/train $DATA_DIR/test; fi
%cd /content/tao_toolkit_recipes/tao_action_recognition/data_generation/
!python3 ./split_dataset.py $DATA_DIR/processed_data $DATA_DIR/splits/testTrainMulti_7030_splits $DATA_DIR/train  $DATA_DIR/test

In [ ]:
# verify
!ls -l $DATA_DIR/train
!ls -l $DATA_DIR/train/ride_bike
!ls -l $DATA_DIR/test
!ls -l $DATA_DIR/test/ride_bike

### 2.2 Download pretrained model from NGC

We will use NGC CLI to get the pre-trained models. For more details, go to https://ngc.nvidia.com and click the SETUP on the navigation bar.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
import os
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 /content/ngccli/ngc-cli/libstdc++.so.6

In [ ]:
!ngc registry model list nvidia/tao/actionrecognitionnet:*

In [ ]:
!mkdir -p $RESULTS_DIR/pretrained

In [ ]:
# Pull pretrained model from NGC 
!ngc registry model download-version "nvidia/tao/actionrecognitionnet:trainable_v1.0" --dest $RESULTS_DIR/pretrained
# Download the optical flow model from NGC
# !ngc registry model download-version "nvidia/tao/actionrecognitionnet:trainable_v2.0" --dest $RESULTS_DIR/pretrained

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $RESULTS_DIR/pretrained/actionrecognitionnet_vtrainable_v1.0

## 3. Provide training specification <a class="anchor" id="head-2"></a>

We provide specification files to configure the training parameters including:

* model_config: configure the model setting
    * model_type: type of model, rgb/of/joint
    * backbone: resnet18/34/50/101/152 
    * rgb_seq_length: length of RGB input sequence
    * input_type: 2d/3d
    * sample_strategy: consecutive
    * dropout_ratio: probability to drop the hidden units
* train_config: configure the training hyperparameters
    * optim_config
    * epochs
    * checkpoint_interval
* dataset_config: configure the dataset and augmentation methods
    * train_dataset_dir
    * val_dataset_dir
    * label_map: map the class label to id
    * output_shape
    * batch_size
    * workers: number of workers to do data loading
    * clips_per_video: number of clips to be sampled from single video
    * augmentation_config

Please refer to the TAO documentation about ActionRecognitionNet to get all the parameters that are configurable.

In [ ]:
!cat $SPECS_DIR/train_rgb_3d_finetune.yaml

## 4. Run TAO training <a class="anchor" id="head-3"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

### 4.1 Train 3D model:

We provide pretrained RGB-only model trained on HMDB5 dataset. With the pretrained model, we can even get better accuracy with less epochs.

`KNOWN ISSUE`: 
- 1) The training log will be corrupted by pytorch warning in the notebook:

     `[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)` 
     
     To see the full log in std out, please run the command in terminal. 
- 2) "=" in the checkpoint file name should removed before using the checkpoint in command.

In [ ]:
print("Train RGB only model with PTM")
!tao action_recognition train \
                  -e $SPECS_DIR/train_rgb_3d_finetune.yaml \
                  -r $RESULTS_DIR/rgb_3d_ptm \
                  -k $KEY \
                  model_config.rgb_pretrained_model_path=$RESULTS_DIR/pretrained/actionrecognitionnet_vtrainable_v1.0/resnet18_3d_rgb_hmdb5_32.tlt  \
                  model_config.rgb_pretrained_num_classes=5

In [ ]:
print('Encrypted checkpoints:')
print('---------------------')
!ls -ltrh $RESULTS_DIR/rgb_3d_ptm

In [ ]:
print('Rename a model: Note that the training is not deterministic, so you may change the model name accordingly.')
print('---------------------')
# NOTE: The following command may require `sudo`. You can run the command outside the notebook.
!find $RESULTS_DIR/rgb_3d_ptm/ -name *epoch=19* | xargs realpath | xargs -I {} mv {} $RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt 
!ls -ltrh $RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt

### `OPTIONAL` 4.2 Train optical flow only model

`Important Note` The following cells are using optical flow dataset. 

We will train a 3D OF-only model

In [ ]:
# print("Train 3D OF-only model")
# !tao action_recognition train \
#                   -e $SPECS_DIR/train_of_3d_finetune.yaml \
#                   -r $RESULTS_DIR/of_3d_ptm \
#                   -k $KEY \
#                   dataset_config.train_dataset_dir=$DATA_DIR/train \
#                   dataset_config.val_dataset_dir=$DATA_DIR/test \
#                   model_config.of_pretrained_model_path=$RESULTS_DIR/pretrained/actionrecognitionnet_vtrainable_v2.0/resnet18_3d_of_hmdb5_32_a100.tlt  \
#                   model_config.of_pretrained_num_classes=5

In [ ]:
# print("To resume training from a checkpoint, set the resume_training_checkpoint_path option to be the .tlt you want to resume from")
# print("remember to remove the `=` in the checkpoint's file name")
# !tao action_recognition train \
#                   -e $SPECS_DIR/train_of_3d.yaml \
#                   -r $RESULTS_DIR/of_3d_ptm \
#                   -k $KEY \
#                   resume_training_checkpoint_path=

In [ ]:
# print('Encrypted checkpoints:')
# print('---------------------')
# !ls -ltrh $HOST_RESULTS_DIR/of_3d_ptm

In [ ]:
# print('Rename a model: ')
# print('---------------------')
# # NOTE: The following command may require `sudo`. You can run the command outside the notebook.
# !find $RESULTS_DIR/of_3d_ptm/ -name *epoch=19* | xargs realpath | xargs -I {} mv {} $RESULTS_DIR/of_3d_ptm/of_only_model.tlt 
# !ls -ltrh $RESULTS_DIR/of_3d_ptm/of_only_model.tlt

## 5. Evaluate trained models <a class="anchor" id="head-4"></a>

We provide two different sample strategy to evaluate the pretrained model on video clips.

* `center` mode: pick up the middle frames of a sequence to do inference. For example, if the model requires 32 frames as input and a video clip has 128 frames, then we will choose the frames from index 48 to index 79 to do the inference. 
* `conv` mode: convolutionly sample 10 sequences out of a single video and do inference. The final results are averaged.

Evaluate RGB model trained with PTM

In [ ]:
!tao action_recognition evaluate \
                    -e $SPECS_DIR/evaluate_rgb.yaml \
                    -k $KEY \
                    model=$RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt  \
                    batch_size=1 \
                    test_dataset_dir=$DATA_DIR/test \
                    video_eval_mode=center

`Optional:` Evaluate OF model

In [ ]:
# !tao action_recognition evaluate \
#                     -e $SPECS_DIR/evaluate_of.yaml \
#                     -k $KEY \
#                     model=$RESULTS_DIR/of_3d_ptm/of_only_model.tlt  \
#                     batch_size=1 \
#                     test_dataset_dir=$DATA_DIR/test \
#                     video_eval_mode=center

## 6. Inferences <a class="anchor" id="head-5"></a>
In this section, we run the action recognition inference tool to generate inferences with the trained RGB models and print the results. 

There are also two modes for inference just like evaluation: `center` mode and `conv` mode. And the final output will show each input sequence label in the videos like:
`[video_sample_path] [labels list for sequences in the video sample]`

In [ ]:
!tao action_recognition inference \
                    -e $SPECS_DIR/infer_rgb.yaml \
                    -k $KEY \
                    model=$RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt \
                    inference_dataset_dir=$DATA_DIR/test/ride_bike \
                    video_inf_mode=center

`Optional:` Inference with OF-only model

In [ ]:
# !tao action_recognition inference \
#                     -e $SPECS_DIR/infer_of.yaml \
#                     -k $KEY \
#                     model=$RESULTS_DIR/of_3d_ptm/of_only_model.tlt \
#                     inference_dataset_dir=$DATA_DIR/test/ride_bike \
#                     video_inf_mode=center